<h1>CS689A: Computational Linguistics for Indian Languages</h1>
<h2>Assignment-2</h2>
<h3>Question-2</h3>

**Dependencies:** `numpy, conllu, datasets, transformers, seqeval`

In [ ]:
!pip install conllu
!pip install datasets

In [ ]:
# !pip install --no-cache-dir transformers sentencepiece
!pip install transformers sentencepiece
!pip install seqeval

### Creating train-validation-test splits
> Replace the file accordingly

**Note: Skip this section if files are already available in form of train-val-test splits**

In [ ]:
from conllu import parse

data_file = open("hindi.conllu", "r", encoding="utf-8",errors='ignore').read()
sentences = parse(data_file)
len(sentences)

13304

In [ ]:
sentences[0] # type TokenList

TokenList<यह, एशिया, की, सबसे, बड़ी, मस्जिदों, में, से, एक, है, ।, metadata={sent_id: "train-s1", text: "यह एशिया की सबसे बड़ी मस्जिदों में से एक है ।"}>

In [ ]:
sentences = [sentence.serialize() for sentence in sentences]

In [ ]:
print(sentences[10]) #type string

# sent_id = train-s11
# text = इसे चार्ल्स कोरिया ने डिजाइन किया है ।
1	इसे	यह	PRON	PRP	Case=Acc,Dat|Number=Sing|Person=3|PronType=Prs	6	obj	_	Vib=को|Tam=ko|ChunkId=NP|ChunkType=head|Translit=ise
2	चार्ल्स	चार्ल्स	NOUN	NNC	Case=Nom|Gender=Masc|Number=Sing|Person=3	3	compound	_	Vib=0|Tam=0|ChunkId=NP2|ChunkType=child|Translit=cārlsa
3	कोरिया	कोरिया	PROPN	NNP	Case=Acc|Gender=Masc|Number=Sing|Person=3	6	nsubj	_	Vib=0_ने|Tam=0|ChunkId=NP2|ChunkType=head|Translit=koriyā
4	ने	ने	ADP	PSP	AdpType=Post	3	case	_	ChunkId=NP2|ChunkType=child|Translit=ne
5	डिजाइन	डिजाइन	NOUN	NN	Case=Nom|Gender=Masc|Number=Sing|Person=3	6	compound	_	Vib=0|Tam=0|ChunkId=NP3|ChunkType=head|Translit=ḍijāina
6	किया	कर	VERB	VM	Aspect=Perf|Gender=Masc|Number=Sing|Person=3|VerbForm=Part|Voice=Act	0	root	_	ChunkId=VGF|ChunkType=head|Stype=declarative|Tam=yA|Translit=kiyā|Vib=या_है
7	है	है	AUX	VAUX	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	6	aux	_	Vib=है|Tam=hE|ChunkId=VGF|ChunkType=child|Translit=hai
8	।	।	PUNCT

In [ ]:
from sklearn.model_selection import train_test_split

## Split size may be adjusted here
train, test = train_test_split(sentences,test_size=0.20)
train, val = train_test_split(train,test_size=0.10)

In [ ]:
len(train),len(val),len(test)

(9578, 1065, 2661)

> Storing the outputs in `data/` folder

In [ ]:
# Here creating train, validation, test conllu files from the original sanskrit conllu file

In [ ]:
import os

## Create directory data/ if does not exist
if not os.path.exists('data'):
    os.makedirs('data')
    
for l,s in zip([train,val,test],['train','validation','test']):
    with open(f'data/{s}.conllu','w',encoding='utf-8') as fp:
        fp.write(''.join(l))

### Tokenization

The following section does tokenization for UPOS tagging task

> Loading data

In [ ]:
from datasets import load_dataset

**Note: If the above convention for saving files was not followed, the file `ConlluIndic.py` needs to be editied so that `__TRAINING_FILE__` etc. point correctly**

In [ ]:
dataset = load_dataset('ConlluIndic.py')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset conllu_indic downloaded and prepared to /root/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/50260ca812993370c93c5949d08ad1fb388e0fced25f584d6ede7b09423bc32c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 9578
    })
    validation: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 1065
    })
    test: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 2661
    })
})

In [ ]:
# for i,j in zip(dataset['train']['tokens'],dataset['train']['upos_tags']):
#   print(i,j)

print(dataset['train']['tokens'][0], dataset['train']['upos_tags'][0])

['हालांकि', 'विशेषाधिकार', 'कानून', 'को', 'पूरी', 'तरह', 'से', 'हटाने', 'की', 'मांग', 'कर', 'रहे', 'अपुंबा', 'लुप', 'ने', 'इस', 'कानून', 'में', 'किसी', 'भी', 'संशोधन', 'को', 'स्वीकार', 'करने', 'से', 'इन्कार', 'कर', 'दिया', '।'] [13, 7, 7, 1, 0, 7, 1, 15, 1, 7, 15, 3, 11, 11, 1, 5, 7, 1, 10, 9, 7, 1, 7, 15, 1, 7, 15, 3, 12]


> Tokenization

In [ ]:
from transformers import AutoTokenizer
import transformers
from transformers import AutoModel, AutoTokenizer
# model_checkpoint = 'ai4bharat/indic-bert'
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=False)

# tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
tokenizer = transformers.AutoTokenizer.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            #if label % 2 == 1:
            #    label += 1
            new_labels.append(label)

    return new_labels


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=False, is_split_into_words=True
    )
    all_labels = examples["upos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets.save_to_disk('data/tokenized_datasets/')

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9578
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1065
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2661
    })
})

In [ ]:
print(tokenized_datasets['train']['labels'][0])

[-100, 13, 7, 7, 7, 7, 7, 7, 1, 0, 7, 1, 15, 15, 1, 7, 15, 3, 11, 11, 11, 1, 5, 7, 7, 1, 10, 9, 7, 7, 7, 1, 7, 7, 15, 1, 7, 7, 15, 3, 3, 12, -100]


### Fine-tuning and Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric, load_from_disk, load_dataset
import numpy as np

> Loads the model, tokenizer, data collator, metric and tokenized datasets

**Note: Don't forget to edit the file paths if the previous conventions were not followed**

In [ ]:
model_checkpoint = 'ai4bharat/indic-bert'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = load_from_disk('data/tokenized_datasets/')



data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = load_metric('seqeval')


dataset = load_dataset('ConlluIndic.py')

upos_feature = dataset['train'].features['upos_tags']
label_names = upos_feature.feature.names



id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}


model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,label2id=label2id)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

In [ ]:
upos_feature.feature.names

['ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 'X',
 '_']

> Metric computation

In [ ]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


> Defining the trainer

`fp16 = True` may be set for GPU

Similarly `batch_size` can be increased

In [ ]:
batch_size = 20
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",
    learning_rate=10e-5,
    num_train_epochs=100,
    weight_decay=0.01,
    save_total_limit=1,
    fp16= True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


Using cuda_amp half precision backend


> Train the model

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9578
  Num Epochs = 100
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 47900
  Number of trainable parameters = 32866834
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.313497,0.866454,0.890454,0.878290,0.901402
2,0.569300,0.247598,0.897779,0.913735,0.905687,0.924354
3,0.240300,0.211155,0.910553,0.921010,0.915752,0.933312
4,0.159800,0.201458,0.918695,0.929533,0.924082,0.938669
5,0.114300,0.223656,0.919384,0.927506,0.923427,0.937176
6,0.089600,0.233432,0.918149,0.925116,0.921619,0.935888
7,0.069300,0.229638,0.919730,0.927090,0.923395,0.936034
8,0.056900,0.236599,0.925051,0.934522,0.929762,0.940865
9,0.046300,0.229455,0.927634,0.932599,0.930110,0.942446
10,0.039300,0.277489,0.920762,0.926934,0.923838,0.935859


***** Running Evaluation *****
  Num examples = 1065
  Batch size = 20
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PART seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/py

TrainOutput(global_step=47900, training_loss=0.019103968213113037, metrics={'train_runtime': 4595.3418, 'train_samples_per_second': 208.428, 'train_steps_per_second': 10.424, 'total_flos': 2982093266314896.0, 'train_loss': 0.019103968213113037, 'epoch': 100.0})

> Evaluate the model

**Note: Make sure evaluation results are reported on test dataset, for this training args should be changed**

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

***** Running Evaluation *****
  Num examples = 2661
  Batch size = 20


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SCONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserW

{'eval_loss': 0.5430908799171448,
 'eval_precision': 0.9448654563138272,
 'eval_recall': 0.945095257397649,
 'eval_f1': 0.9449803428849349,
 'eval_accuracy': 0.9482500680025387,
 'eval_runtime': 9.5555,
 'eval_samples_per_second': 278.479,
 'eval_steps_per_second': 14.023,
 'epoch': 100.0}

In [ ]:
{'eval_loss': 0.5430908799171448,
 'eval_precision': 0.9448654563138272,
 'eval_recall': 0.945095257397649,
 'eval_f1': 0.9449803428849349,
 'eval_accuracy': 0.9482500680025387,
 'eval_runtime': 9.5555,
 'eval_samples_per_second': 278.479,
 'eval_steps_per_second': 14.023,
 'epoch': 100.0}